In [1]:
#Loading dependencies
import numpy as np
import pandas as pd
import os
import pickle
from tqdm import tqdm
import re
from scipy.spatial.distance import cdist
from itertools import combinations, product
import os, fnmatch
from biopandas.pdb import PandasPdb


In [2]:
path = '/Users/eric1999/Desktop/Eric/'
os. chdir(path)

#Paths of the ML models and where to read the pdbs and store them
pathML = '/Users/eric1999/Desktop/ML-stacking-noh_Eric/caca/'
pathPDB = '/Users/eric1999/Desktop/Eric/'


In [3]:
## Define functions
def potency2(X):
#    X3 = [ 1/(x**3) for x in X ]
    X6 = [ 1/(x**6) for x in X ]
#    X9 = [ 1/(x**9) for x in X ]
#    X10 = [ 1/(x**10) for x in X ]
    X12 = [ 1/(x**12) for x in X ]
    X = [ 1/x for x in X ]
#    out = np.array([X + X3 + X6 + X9 + X10 + X12])
    out = np.array([X + X6 + X12])
    return out

def parsexyz(pdbfile, path):
    x = {}
    y = {}
    z = {}
    count = 0
    for line in open(os.path.join(os.path.abspath(path), pdbfile)):
        splited = line.split()
        id = splited[0]
        if id == "ATOM":
            x[count] = float(splited[6])
            y[count] = float(splited[7])
            z[count] = float(splited[8])
            count = count + 1
    #Convert into dataframe
    df = {'x':x, 'y':y, 'z':z}
    df = pd.DataFrame(data=df)
    return df

def parsepdb(pdbfile, path):
    eleno = {}
    elety = {}
    resid = {}
    resno = {}
    x = {}
    y = {}
    z = {}
    B_factor = {}
    count = 0
    for line in open(os.path.join(os.path.abspath(path), pdbfile)):
        splited = line.split()
        id = splited[0]
        if id == "ATOM":
            eleno[count] = int(splited[1])
            elety[count] = splited[2]
            resid[count] = splited[3]
            resno[count] = int(splited[5])
            x[count] = float(splited[6])
            y[count] = float(splited[7])
            z[count] = float(splited[8])
            B_factor[count] = float(splited[9])
            count = count + 1
    #Convert into dataframe
    df = {'eleno': eleno, 'elety': elety, 'resid': resid, 'resno':resno, 'x':x, 'y':y, 'z':z, 'B_factor':B_factor}
    df = pd.DataFrame(data=df)
    df['resid'] = df['resid'].replace("5", "", regex=True)
    df['resid'] = df['resid'].replace("3", "", regex=True)
    return df

#Generate dataframe and bind energies

def scoring_backbone(top, energies, pdb):
    df = pd.DataFrame(top[0],columns=['Atom1', 'Atom2'])
    df["Energies"] = energies

    #Generate identifiers
    Identifiers = np.unique(list(df["Atom1"]) +  list(df["Atom2"]))
    
    #Sum the energies per base
    TotalEnergiesPerBase = []

    for i in Identifiers:
        Summed = sum(df[(df["Atom1"] == i) | (df["Atom2"] == i)]["Energies"])
        TotalEnergiesPerBase.append(Summed)
    
    d = {'Atoms':Identifiers,'TotalEnergies':TotalEnergiesPerBase}
    df_Final = pd.DataFrame(d)
    
    normalized = (df_Final["TotalEnergies"] - min(df_Final["TotalEnergies"])) / (max(df_Final["TotalEnergies"]) - min(df_Final["TotalEnergies"]))
    
    df_Final["TotalEnergies"] = normalized
    
    for i in df_Final["Atoms"]:
        energy_rep = [df_Final[df_Final["Atoms"] == i]["TotalEnergies"]] * len(pdb.loc[pdb["resno"] == i, "B_factor"])
        pdb.loc[pdb["resno"] == i, "B_factor"] = energy_rep
    
    return(pdb)

def topology(pdbfile, path):
    df = parsepdb(pdbfile=pdbfile, path=path)
    ## Get data for C1' atom of all residues
    mindf = df[df.elety == "C4"]
    ## Get combinations of all against all atoms
    combs_close = list(combinations(mindf.resno, 2))
    drop = df['elety'].isin(to_drop)
    df2 = df[~drop]
    df2.index = range(0, len(df2.index))
    sorterA=["C1'", "N9", "C8", "N7", "C5", "C6", "N6","N1", "C2", "N3", "C4"]
    sorterC=["C1'", "N1", "C6", "C5", "C4", "N4","N3", "C2", "O2"]
    sorterG=["C1'", "N9", "C8", "N7", "C5", "C6", "O6", "N1","C2", "N2","N3", "C4"]
    sorterU=["C1'", "N1", "C6","C5","C4", "O4", "N3","C2", "O2"]
    sorterIndexA = dict(zip(sorterA,range(len(sorterA))))
    sorterIndexC = dict(zip(sorterC,range(len(sorterC))))
    sorterIndexG = dict(zip(sorterG,range(len(sorterG))))
    sorterIndexU = dict(zip(sorterU,range(len(sorterU))))
    pairs=[]
    ix=[]
    combs=[]
    for i in combs_close:
        ## Get residue names of the pair
        resid1 = mindf.loc[mindf['resno'] == i[0], ['resid']].to_numpy()
        resid2 = mindf.loc[mindf['resno'] == i[1], ['resid']].to_numpy()
        pair = str(resid1 + resid2)[3:5]
        ## If not recognized, swap it
        if not pair in listofpairs:
            i = (i[1], i[0])
            resid1 = mindf.loc[mindf['resno'] == i[0], ['resid']].to_numpy()
            resid2 = mindf.loc[mindf['resno'] == i[1], ['resid']].to_numpy()
            pair = str(resid1 + resid2)[3:5]
        ## Save pair
        pairs.append(pair)
        ## Get indices for the first base according with pair and desired order of atoms
        if resid1 == 'A':
            sorterIndex = sorterIndexA
        elif resid1 == 'C':
            sorterIndex = sorterIndexC
        elif resid1 == 'G':
            sorterIndex = sorterIndexG
        elif resid1 == 'U':
            sorterIndex = sorterIndexU
        #data_base_1=df2[df2.resno == i[0]]
        #ix1=data_base_1.iloc[data_base_1['elety'].map(sorterIndex),].index
        data_base_1=df2[df2.resno == i[0]]
        data_base_1.insert(len(data_base_1.columns), 'rank', data_base_1['elety'].map(sorterIndex))
        ix1=data_base_1.sort_values(['rank']).index
        ## Get indices for the second base according with pair and desired order of atoms
        if resid2 == 'A':
            sorterIndex = sorterIndexA
        elif resid2 == 'C':
            sorterIndex = sorterIndexC
        elif resid2 == 'G':
            sorterIndex = sorterIndexG
        elif resid2 == 'U':
            sorterIndex = sorterIndexU
        data_base_2=df2[df2.resno == i[1]]
        data_base_2.insert(len(data_base_2.columns), 'rank', data_base_2['elety'].map(sorterIndex))
        ix2=data_base_2.sort_values(['rank']).index
        #ix1=data_base_1.index[data_base_1.resno == i[0]]
        #ix2=data_base_2.index[data_base_2.resno == i[1]]
        ix.append([ix1, ix2])
        combs.append(list(product(ix1, ix2)))
    return [combs_close, pairs, ix, combs, drop]

def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

## Define additional dependencies
to_drop = ["P", "OP1", "OP2", "OP3", "HO5'", "O5'", "C5'", "H5'", "H5''", "C4'", "H4'", "O4'", "H1'", "C3'", 
           "H3'", "C2'", "H2'", "O2'", "HO2'", "O3'", "HO3'", "H11'", "H12'", "H13'",
          "H8", "H61", "H62", "H2", "H6",  "H5", "H41", "H42",  "H1","H21","H22", "H3"]
to_keep = ["N1", "C2", "N3", "C4", "C5", "C6"]
listofpairs = ['AA', 'AC', 'AG', 'AU', 'CC', 'CU', 'GC', 'GG', 'GU', 'UU']

## Define model filenames
#AA_modelfile = pathML + '/AA_model_tpot.sav'
#AC_modelfile = pathML + '/AC_model_tpot.sav'
#AG_modelfile = pathML + '/AG_model_tpot.sav'
#AU_modelfile = pathML + '/AU_model_tpot.sav'
#CC_modelfile = pathML + '/CC_model_tpot.sav'
#CU_modelfile = pathML + '/CU_model_tpot.sav'
#GC_modelfile = pathML + '/GC_model_tpot.sav'
#GG_modelfile = pathML + '/GG_model_tpot.sav'
#GU_modelfile = pathML + '/GU_model_tpot.sav'
#UU_modelfile = pathML + '/UU_model_tpot.sav'
AA_modelfile = pathML + '/AA/model_scanned.sav'
AC_modelfile = pathML + '/AC/model_scanned.sav'
AG_modelfile = pathML + '/AG/model_scanned.sav'
AU_modelfile = pathML + '/AU/model_scanned.sav'
CC_modelfile = pathML + '/CC/model_scanned.sav'
CU_modelfile = pathML + '/CU/model_scanned.sav'
GC_modelfile = pathML + '/GC/model_scanned.sav'
GG_modelfile = pathML + '/GG/model_scanned.sav'
GU_modelfile = pathML + '/GU/model_scanned.sav'
UU_modelfile = pathML + '/UU/model_scanned.sav'

# Read models from disk
AA_model = pickle.load(open(AA_modelfile, 'rb'))
AC_model = pickle.load(open(AC_modelfile, 'rb'))
AG_model = pickle.load(open(AG_modelfile, 'rb'))
AU_model = pickle.load(open(AU_modelfile, 'rb'))
CC_model = pickle.load(open(CC_modelfile, 'rb'))
CU_model = pickle.load(open(CU_modelfile, 'rb'))
GC_model = pickle.load(open(GC_modelfile, 'rb'))
GG_model = pickle.load(open(GG_modelfile, 'rb'))
GU_model = pickle.load(open(GU_modelfile, 'rb'))
UU_model = pickle.load(open(UU_modelfile, 'rb'))

In [10]:
files = fnmatch.filter(os.listdir(pathPDB), '*.pdb')
files = sorted(files)

In [11]:
for index in range(len(files)):
    
    pdbfile = files[index]
    print(pdbfile)
    #Gather topology
    top = topology(pdbfile=pdbfile, path=pathPDB)
    
    #Obtain distances and compute energies
    df = parsexyz(pdbfile = pdbfile, path = pathPDB)
    df = df[~top[4]]
    ## Calculate all distances between atoms
    dists=cdist(df.iloc[:,], df.iloc[:,], metric='euclidean')
    combs_close = top[0]
    myseq = list(range(0, len(combs_close), 1))
    energies = []
    for i in myseq:
        pair = top[1][i]
        ## Substract desired distances based on presaved indices
        distances=dists[top[2][i][0],:][:,top[2][i][1]].flatten()

        #Check if distances are lower than 1.5A or higher than 10A
        check_low  = np.any(distances < 1.5)
        check_up = np.any(distances > 10)

        ## Calculate powers
        dd = potency2(distances).reshape(1, -1)
        modelname = str(pair + '_model.predict(dd)')
        #modelname = modelname[3:23]
        #print(modelname)
        out = round(eval(modelname)[0], 6)
        #print(top[0][i], pair, out)
        #print(pair)
        energies.append(out)
    out=round(sum(energies), 4)
    
    print(pdbfile)
    print(out)
    print("#########")
    
    pdb = parsepdb(pdbfile=pdbfile, path=pathPDB)
    
    #Scoring
    scored_pdb = scoring_backbone(top = top, energies = energies, pdb = pdb)
    
    #Read the pdb
    pl1 = PandasPdb().read_pdb(pathPDB + pdbfile)

    pl1.df['ATOM']["b_factor"] = scored_pdb["B_factor"]
    
    
    pdbfile_score = pdbfile.split('.pdb')
    pdbfile_score = pdbfile_score[0] + '_score.pdb'
    path_score = pathPDB + pdbfile_score
    
    #Store pdb
    pl1.to_pdb(path= path_score, 
                records=None, 
                gz=False, 
                append_newline=True)
    




structure_000.pdb
structure_000.pdb
604.6065
#########
structure_001.pdb
structure_001.pdb
580.5313
#########
structure_002.pdb
structure_002.pdb
528.8363
#########
structure_003.pdb
structure_003.pdb
366.0133
#########
structure_004.pdb
structure_004.pdb
352.8935
#########
structure_005.pdb
structure_005.pdb
71.6559
#########
structure_006.pdb
structure_006.pdb
68.187
#########
structure_007.pdb
structure_007.pdb
69.474
#########
structure_008.pdb
structure_008.pdb
66.434
#########
structure_009.pdb
structure_009.pdb
62.6831
#########
structure_010.pdb
structure_010.pdb
-439.8154
#########
structure_011.pdb
structure_011.pdb
-439.3496
#########
structure_012.pdb
structure_012.pdb
-439.4183
#########
structure_013.pdb
structure_013.pdb
-479.9186
#########
structure_014.pdb
structure_014.pdb
-477.8913
#########
structure_015.pdb
structure_015.pdb
-473.5912
#########
structure_016.pdb
structure_016.pdb
-1333.0928
#########
structure_017.pdb
structure_017.pdb
239.1986
#########
structure_